<a href="https://colab.research.google.com/github/Huvinesh-Rajendran-12/Computer-Vision/blob/main/050521_ComputerVision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Operations

## Import Libraries

In [ ]:
import numpy as np 
import pandas as pd 
import cv2 as cv 
from google.colab.patches import cv2_imshow 
from skimage import io 
from PIL import Image 
import matplotlib.pylab as plt 


In [ ]:
data_dir = "/content/drive/MyDrive/Practical/Computer Vision and NLp/tom-coe-FXhRrfFH9kc-unsplash.jpg"
image = cv.imread(data_dir)
cv2_imshow(image)
cv.waitKey(0)

In [ ]:
# Check the image matrix data type (could know the bit depth of the image)
print(image.dtype)
# Check the height of image 
print(image.shape[0])
# Check the width of image 
print(image.shape[1])
# Check the number of channels of the image
print(image.shape[2])

In [ ]:
print(image.shape)

## Resizing(Scaling) 

In [ ]:
# to prevent image does from skewed or distorted 
# calculate the ratio of the new image to the old image
r = 500 / image.shape[1]
dim = (500, int(image.shape[0] * r))

# perform the actual resizing of the image and show it
resized = cv.resize(image, dim, interpolation = cv.INTER_AREA)
cv2_imshow(resized)
cv.waitKey(0)

## Rotating Image

In [ ]:
# grab the dimensions of the image and calculate the center
(h, w) = resized.shape[:2]
center = (w / 2, h / 2)

# rotate the image by 180 degrees
M = cv.getRotationMatrix2D(center, 180, 1.0)
rotated = cv.warpAffine(resized, M, (w, h))
cv2_imshow(rotated)

# re-rotate
M = cv.getRotationMatrix2D(center, 180, 1.0)
rotated = cv.warpAffine(rotated, M, (w, h))
cv2_imshow(rotated)
cv.waitKey(0)

## Histogram

In [ ]:
plt.hist(rotated.ravel(),bins = 256, range = [0,256]) 
plt.show()

### Display the histogram of RGB channel

In [ ]:
color = ('r','g','b')
for i,col in enumerate(color):
    histr = cv.calcHist([rotated],[i],None,[256],[0,256])
    plt.plot(histr,color = col)
    plt.xlim([0,256])
plt.show()

In [ ]:
gray_image = cv.cvtColor(rotated, cv.COLOR_RGB2GRAY)
cv2_imshow(gray_image)

In [ ]:
# Plot the histogram of the gray image. We could observe that the frequency of
# the image hist has decreased ~ 1/3 of the histogram of color image
plt.hist(gray_image.ravel(),bins = 256, range = [0, 256])
plt.show()

## Grayscale Transformation 

In [ ]:
# This is an inverse operation of the grayscale image, you could see that the
# bright pixels become dark, and the dark pixels become bright
im2 = 255 - gray_image
cv2_imshow(im2)

In [ ]:
# Another transform of the image, after adding a constant, 
# all the pixels become brighter and a hazing-like effect of the image is generated
im3 = (100/255)*gray_image + 100
cv2_imshow(im3)

In [ ]:
# The lightness level of the gray_image decreases after this step
im4 = 255*(gray_image/255)**2
cv2_imshow(im4)
plt.hist(im4.ravel(),bins = 256, range = [0, 256])
plt.show()

## Histogram Equalization 

In [ ]:
# function of histogram equalization
def histeq(im, nbr_bins = 256):
  """ Histogram equalization of a grayscale image.  """
  # get the image histogram
  imhist, bins = np.histogram(im.flatten(), nbr_bins, [0, 256])
  cdf = imhist.cumsum() # cumulative distribution function
  cdf = imhist.max()*cdf/cdf.max()  #normalize
  cdf_mask = np.ma.masked_equal(cdf, 0)
  cdf_mask = (cdf_mask - cdf_mask.min())*255/(cdf_mask.max()-cdf_mask.min())
  cdf = np.ma.filled(cdf_mask,0).astype('uint8')
  return cdf[im.astype('uint8')]

# apply the function on your dark image to increase the contrast
# we could observe that the contrast of the black background has increased
im5 = histeq(im4)
cv2_imshow(im5)

In [ ]:
# Extra: try to visualize the histogram of the image after histogram equalization
# Before histogram equalization
plt.hist(im4.ravel(),bins = 256, range = [0, 256])
plt.show()

In [ ]:
# After histogram equalization
plt.hist(im5.ravel(),bins = 256, range = [0, 256])
plt.show()

## Dilation & Erosion

In [ ]:
# Python program to demonstrate erosion & dilation of images.
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
  
# Reading the input image
img = io.imread("https://placekitten.com/800/571") 

# to prevent image does from skewed or distorted 
# calculate the ratio of the new image to the old image
r = 300 / img.shape[1]
dim = (300, int(img.shape[0] * r))

# Warning: Color image loaded by OpenCV is in BGR mode. 
# But Matplotlib displays in RGB mode. 
# So color images will not be displayed correctly in Matplotlib 
# if image is read with OpenCV.
bgr2rgb = cv2.cvtColor(img, cv.COLOR_BGR2RGB)
cv2_imshow(bgr2rgb)

# perform the actual resizing of the image and show it
resized3 = cv.resize(bgr2rgb, dim, interpolation = cv.INTER_AREA)
cv2_imshow(resized3)
cv.waitKey(0)

print(resized3.shape)

# crop the image using array slices 
cropped = resized3[100:140, 100:140]
cv2_imshow(cropped)
cv.waitKey(0)

gray_image3 = cv.cvtColor(resized3, cv.COLOR_BGR2GRAY)
cv2_imshow(gray_image3)
# Taking a matrix of size 5 as the kernel
kernel = np.ones((5,5), np.uint8)
  
# The first parameter is the original image,
# kernel is the matrix with which image is convolved 
# third parameter is the number of iterations, 
# to determine how much you want to erode/dilate a given image. 
img_erosion2 = cv2.erode(gray_image3, kernel, iterations=1)
img_dilation2 = cv2.dilate(gray_image3, kernel, iterations=1)

cv2_imshow(img_erosion2)
cv2_imshow(img_dilation2)
  
cv2.waitKey(0)

## Save the cropped image

In [ ]:
cv2.imwrite("/content/drive/MyDrive/Colab Notebooks/crooped_nose.png", cropped)